### Update information: Input required
#### Note: quotation marks should be included. Change values inside of the quotes.

In [1]:
AY = '2022-2023'
CO = 'c/o 2023'
CS = 'Pediatrics EM'

That's it. Now you can sit back and run the code! 
### Course Data Automation

In [2]:
# initiate
import numpy as np
import pandas as pd
from datetime import datetime
import datetime
import time
import warnings
warnings.simplefilter("ignore")

# start timer
tic = time.time()

# import dataset
read = pd.read_excel('input.xlsx', engine='openpyxl')

#filter in applicable data only: "0/IP Address" rows only
readfile = read[(read.Status.isin(['Response Type', 0]))]

# subset site survey questions
q_subset = readfile.loc[:,readfile.columns.str.contains('.*Q.*')]

# create a list of columns
q_cols_subset = list(q_subset.columns)

# calculate the number of one set of questions
set_n = sum(1 for i in q_cols_subset if i[0:5] == (q_cols_subset[0])[0:5])

# create a placeholder dataframe
df0 = pd.DataFrame(columns = ['AcademicYear', 'Class', 'Course' ,'Type'])
#df0

# create a dataframe for stats
def stat_df(subset_cols_df, subset_df):
    
    stat_df1 = pd.DataFrame([])
    qn=0
    for j in subset_cols_df:

        c = subset_df[j][1:,].value_counts().rename_axis('unique_values').reset_index(name='counts')
        c['pct'] = c.counts / c.counts.sum()



        if (qn == set_n):
            qn = 1

        else:
            qn +=1

        c['QN'] = qn

        cc = {}
        for i in range(1,6):
            if i in list(c.unique_values):
                cn = c[c.unique_values == i]['counts']
                pn = c[c.unique_values == i]['pct']
                cc.update(                {
                        'C'+str(i):int(cn), 'R'+str(i):float(pn)
                    }            )

            else:
                cc.update(                {
                        'C'+str(i):0, 'R'+str(i):0
                    }              )



        dd = pd.DataFrame([cc])
        dd['N'] = dd.C1 + dd.C2 + dd.C3 + dd.C4 + dd.C5
        dd['A_SA'] = dd.R4 + dd.R5
        dd['Sumproduct'] = dd.C1 * 1 + dd.C2 * 2 + dd.C3 * 3 + dd.C4 * 4 + dd.C5 * 5
        dd['Mean'] = (dd.C1 * 1 + dd.C2 * 2 + dd.C3 * 3 + dd.C4 * 4 + dd.C5 * 5)/dd.N
        dd['QN']= c['QN']
        stat_df1 = pd.concat([stat_df1, dd], axis=0, ignore_index=True)
        stat_df2 = stat_df1[['QN','Sumproduct',
                            'R1', 'R2', 'R3', 'R4', 'R5', 'A_SA', 'Mean', 'N', 'C1', 'C2', 'C3', 'C4', 'C5']]
    return stat_df2

df1 = stat_df(q_cols_subset, q_subset)
#df1

# create a dataframe for question by extracting text
sequence2=0
q0 = []
for l in q_cols_subset:
    q_text_q = str(q_subset[l][0])  
    q0.append(q_text_q)


df_q1 = pd.DataFrame(q0)
df_q1.columns = ['Question']
#df_q1

# concat dataframes
df_s1 = pd.concat([df_q1, df1], axis=1)
df4 = pd.concat([df0, df_s1], axis=1)
#df4

# delete unapplicable rows/qualitative questions
df5 = df4[df4['N'] > 0]

# populate columns for AY, Class, Block
df5 = df5.assign(AcademicYear = AY)
df5 = df5.assign(Class = CO)
df5 = df5.assign(Course = CS)
df5 = df5[['AcademicYear', 'Class', 'Course', 'Question',
          'R1', 'R2', 'R3', 'R4', 'R5', 'A_SA', 'Mean', 'N', 'C1', 'C2', 'C3', 'C4', 'C5']]
#df5

# export
df5.to_excel('output_' + str(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")) + '.xlsx', index = False)

# end timer             
toc = time.time() - tic
print('All tasks completed in %.2f s.' % (toc))

All tasks completed in 0.73 s.
